In [ ]:
# 1. Setup & Configuration
# Imports, constants, schema definitions

import os
import io
import re
from datetime import datetime, timedelta, date
import pandas as pd
import numpy as np

# For accessing Colab secrets
from google.colab import userdata

# Environment variables (recommended to set externally)
STAGE1_USER = userdata.get("STAGE1_USER")
STAGE1_PWD = userdata.get("STAGE1_PWD")
STAGE2_USER = userdata.get("STAGE2_USER")
STAGE2_PWD = userdata.get("STAGE2_PWD")

# Global configuration
TODAY = pd.to_datetime(date.today()).normalize()
CURRENT_MONTH_START_DATE = pd.to_datetime(TODAY.strftime('%Y-%m-01'))

ALLOWED_EXTS = ['.csv', '.xlsx']
MAX_FILE_MB = 50

REQUIRED_CUSTOMER_COLS = ['customer_id', 'customer_name', 'mobile_number', 'region']
REQUIRED_ORDER_COLS = ['order_id', 'mobile_number', 'order_date_time', 'sku_id', 'sku_count', 'total_amount']

# Dataframes
loaded_customers_df = pd.DataFrame()
loaded_orders_df = pd.DataFrame()
temp_previous_day_df = pd.DataFrame()

# History folder
HISTORY_ROOT = '/content/data_history'
os.makedirs(HISTORY_ROOT, exist_ok=True)

print("Environment ready. TODAY =", TODAY.date())

In [ ]:

# 2. Helper Functions: normalization, schema check, save history, error handling

def normalize_colname(c: str) -> str:
    c = str(c).strip().lower()
    c = re.sub(r'[\s]+', '_', c)
    c = re.sub(r'[^a-z0-9_]', '', c)
    c = re.sub(r'__+', '_', c)
    return c

def normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = [normalize_colname(c) for c in df.columns]
    return df

def missing_required_columns(actual_cols, required_cols):
    return [c for c in required_cols if c not in actual_cols]

def read_csv_bytes(file_bytes):
    try:
        return pd.read_csv(io.BytesIO(file_bytes))
    except Exception as e:
        raise ValueError(f"Error reading uploaded file: {e}")

def save_daily_history(customers_df, orders_df, day=TODAY):
    day_str = day.strftime('%Y-%m-%d')
    folder = os.path.join(HISTORY_ROOT, day_str)
    os.makedirs(folder, exist_ok=True)
    cust_path = os.path.join(folder, f'customers_{day_str}.csv')
    orders_path = os.path.join(folder, f'orders_{day_str}.csv')
    if not customers_df.empty:
        customers_df.to_csv(cust_path, index=False)
    if not orders_df.empty:
        orders_df.to_csv(orders_path, index=False)
    return folder


In [ ]:

# 3. Stage-1 Authorization & Upload Customer CSV + Orders XML

from google.colab import files
import getpass
import xml.etree.ElementTree as ET

# Stage-1 authentication
user_input = input("Stage-1 Username: ").strip()
pwd_input = getpass.getpass("Stage-1 Password: ")

if user_input != STAGE1_USER or pwd_input != STAGE1_PWD:
    raise ValueError("Stage-1 Authentication Failed")
print("Stage-1 Authenticated")

# File upload: select BOTH customer CSV/XLSX and order XML
uploaded = files.upload()
if not uploaded:
    raise ValueError("No file uploaded")

# Helper function to parse XML orders
def read_orders_xml(file_bytes):
    try:
        tree = ET.parse(io.BytesIO(file_bytes))
        root = tree.getroot()
        rows = []
        for order in root.findall('.//order'):  # adjust tag as per XML
            row = {
                'order_id': order.findtext('order_id'),
                'mobile_number': order.findtext('mobile_number'),
                'order_date_time': order.findtext('order_date_time'),
                'sku_id': order.findtext('sku_id'),
                'sku_count': int(order.findtext('sku_count') or 0),
                'total_amount': float(order.findtext('total_amount') or 0),
            }
            rows.append(row)
        df = pd.DataFrame(rows)
        return normalize_columns(df)
    except Exception as e:
        raise ValueError(f"Error parsing XML file: {e}")

# Load files
for file_name, fbytes in uploaded.items():
    ext = os.path.splitext(file_name)[1].lower()
    if ext in ['.csv', '.xlsx'] and 'customer' in file_name.lower():
        loaded_customers_df = read_csv_bytes(fbytes)
        loaded_customers_df = normalize_columns(loaded_customers_df)
        missing = missing_required_columns(loaded_customers_df.columns.tolist(), REQUIRED_CUSTOMER_COLS)
        if missing:
            raise ValueError(f"Missing required customer columns: {missing}")
        loaded_customers_df['mobile_number'] = loaded_customers_df['mobile_number'].astype(str).str.strip()
        print(f"Customer CSV loaded. Rows: {len(loaded_customers_df)}")
    elif ext == '.xml' and 'order' in file_name.lower():
        loaded_orders_df = read_orders_xml(fbytes)
        missing_orders = missing_required_columns(loaded_orders_df.columns.tolist(), REQUIRED_ORDER_COLS)
        if missing_orders:
            raise ValueError(f"Missing required order columns: {missing_orders}")
        loaded_orders_df['mobile_number'] = loaded_orders_df['mobile_number'].astype(str)
        loaded_orders_df['total_amount'] = pd.to_numeric(loaded_orders_df['total_amount'])
        loaded_orders_df['order_date_time'] = pd.to_datetime(loaded_orders_df['order_date_time'], utc=True)
        print(f"Orders XML loaded. Rows: {len(loaded_orders_df)}")
    else:
        print(f"Skipping unsupported file: {file_name}")


In [ ]:
# 4. Backup previous day & select current day orders

# Make TODAY and CURRENT_MONTH_START_DATE timezone-aware
TODAY_UTC = pd.to_datetime(date.today()).normalize().tz_localize('UTC')
CURRENT_MONTH_START_DATE_UTC = pd.to_datetime(TODAY.strftime('%Y-%m-01')).tz_localize('UTC')

temp_previous_day_df = loaded_orders_df[loaded_orders_df['order_date_time'] < TODAY_UTC]
current_day_orders_df = loaded_orders_df[loaded_orders_df['order_date_time'] >= TODAY_UTC]

# Fallback to current month if no rows for today
if current_day_orders_df.empty:
    current_day_orders_df = loaded_orders_df[loaded_orders_df['order_date_time'] >= CURRENT_MONTH_START_DATE_UTC]

print("Previous-day rows:", len(temp_previous_day_df))
print("Current-day rows for analysis:", len(current_day_orders_df))

# Save daily history
history_folder = save_daily_history(loaded_customers_df, loaded_orders_df)
print("Daily history saved at:", history_folder)

In [ ]:
# 5. Merge Customers & Orders

merged_df = pd.merge(
    loaded_customers_df,
    current_day_orders_df[['order_id','mobile_number','order_date_time','total_amount']],
    on='mobile_number',
    how='inner'
)
print(f"Merged DataFrame. Rows: {len(merged_df)}")
merged_df.head()


In [ ]:
# 7. KPI Functions & Calculation

def kpi_repeat_customers(df):
    return df.groupby(['customer_id','customer_name']).agg(
        order_count=('order_id','count')
    ).reset_index().query('order_count>1')

def kpi_regional_revenue(df):
    res = df.groupby('region').agg(total_revenue=('total_amount','sum')).reset_index()
    return res.sort_values('total_revenue', ascending=False)

kpi_repeat = kpi_repeat_customers(merged_df)
kpi_region = kpi_regional_revenue(merged_df)

print("Repeat Customers:")
display(kpi_repeat.head())

print("Regional Revenue:")
display(kpi_region.head())


In [ ]:
# 6. All KPI Functions & Calculation

def kpi_repeat_customers(df):
    return df.groupby(['customer_id','customer_name']).agg(
        order_count=('order_id','count')
    ).reset_index().query('order_count>1')

def kpi_regional_revenue(df):
    res = df.groupby('region').agg(total_revenue=('total_amount','sum')).reset_index()
    return res.sort_values('total_revenue', ascending=False)

def kpi_monthly_order_trends(df):
    # Ensure order_date_time is datetime and timezone-aware for comparison
    df['order_date_time'] = pd.to_datetime(df['order_date_time'], errors='coerce', utc=True)
    df = df.dropna(subset=['order_date_time']) # Drop rows with invalid dates
    df['order_month'] = df['order_date_time'].dt.to_period('M')
    return df.groupby('order_month').agg(order_count=('order_id','count')).reset_index().sort_values('order_month')

def kpi_top_customers_last_30_days(df):
    # Ensure order_date_time is datetime and timezone-aware for comparison
    df['order_date_time'] = pd.to_datetime(df['order_date_time'], errors='coerce', utc=True)
    df = df.dropna(subset=['order_date_time']) # Drop rows with invalid dates

    last_30_days_start = TODAY_UTC - timedelta(days=30)
    recent_orders_df = df[df['order_date_time'] >= last_30_days_start]

    return recent_orders_df.groupby(['customer_id', 'customer_name']).agg(
        total_spend=('total_amount', 'sum')
    ).reset_index().sort_values('total_spend', ascending=False)



In [ ]:
#7 Calculate all KPIs

# Merge orders with customers to get customer names for the top customers KPI
orders_with_customer_info = pd.merge(
    loaded_orders_df,
    loaded_customers_df[['mobile_number', 'customer_id', 'customer_name']],
    on='mobile_number',
    how='inner'
)

kpi_repeat = kpi_repeat_customers(merged_df)
kpi_region = kpi_regional_revenue(merged_df)
kpi_monthly_trends = kpi_monthly_order_trends(loaded_orders_df) # Use loaded_orders_df for full history
kpi_top_customers = kpi_top_customers_last_30_days(orders_with_customer_info) # Use the merged dataframe


print("Repeat Customers:")
display(kpi_repeat.head())

print("\nRegional Revenue:")
display(kpi_region.head())

print("\nMonthly Order Trends:")
display(kpi_monthly_trends.head())

print("\nTop Customers by Spend (Last 30 Days):")
display(kpi_top_customers.head())

In [ ]:
# 8. Stage-2 Authorization & Export Excel

user2 = input("Stage-2 Username: ").strip()
pwd2 = getpass.getpass("Stage-2 Password: ")

if user2 != STAGE2_USER or pwd2 != STAGE2_PWD:
    raise ValueError("Stage-2 Authentication Failed")
print("Stage-2 Authorized. Exporting report...")

# Format the 'order_date_time' column as string before writing to Excel
# Check if the column is already in datetime format before formatting
if pd.api.types.is_datetime64_any_dtype(current_day_orders_df['order_date_time']):
    current_day_orders_df['order_date_time'] = current_day_orders_df['order_date_time'].dt.strftime('%Y-%m-%d %H:%M:%S')
if pd.api.types.is_datetime64_any_dtype(temp_previous_day_df['order_date_time']):
    temp_previous_day_df['order_date_time'] = temp_previous_day_df['order_date_time'].dt.strftime('%Y-%m-%d %H:%M:%S')


report_file = f'/content/Daily_Report_{TODAY.strftime("%Y%m%d")}.xlsx'
with pd.ExcelWriter(report_file, engine='openpyxl') as writer:
    loaded_customers_df.to_excel(writer, sheet_name='Customers', index=False)
    current_day_orders_df.to_excel(writer, sheet_name='Current_Day_Orders', index=False)
    temp_previous_day_df.to_excel(writer, sheet_name='Archived_Previous_Day', index=False)
    kpi_repeat.to_excel(writer, sheet_name='KPI_Repeat_Customers', index=False)
    kpi_region.to_excel(writer, sheet_name='KPI_Regional_Revenue', index=False)
    kpi_monthly_trends.to_excel(writer, sheet_name='KPI_Monthly_Order_Trends', index=False)
    kpi_top_customers.to_excel(writer, sheet_name='KPI_Top_Customers_Last_30_Days', index=False)


print("Report exported:", report_file)

# Optional Colab download link
from IPython.display import FileLink
FileLink(report_file)